In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torchsummary
import scripts.models as models

In [2]:
model = models.StormSqueezeNet()

model.to('cuda')
torchsummary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 111, 111]           1,792
         LeakyReLU-2         [-1, 64, 111, 111]               0
         MaxPool2d-3           [-1, 64, 55, 55]               0
            Conv2d-4           [-1, 16, 55, 55]           1,040
         LeakyReLU-5           [-1, 16, 55, 55]               0
       BatchNorm2d-6           [-1, 16, 55, 55]              32
            Conv2d-7           [-1, 64, 55, 55]           1,088
         LeakyReLU-8           [-1, 64, 55, 55]               0
       BatchNorm2d-9           [-1, 64, 55, 55]             128
           Conv2d-10           [-1, 64, 55, 55]           9,280
        LeakyReLU-11           [-1, 64, 55, 55]               0
      BatchNorm2d-12           [-1, 64, 55, 55]             128
        StormFire-13          [-1, 128, 55, 55]               0
           Conv2d-14           [-1, 16,

In [7]:

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decode = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            StormFire(256, 64, 128,128),
            nn.Upsample(scale_factor=2),
            StormFire(128, 32, 64,64),
            nn.Upsample(scale_factor=2),
            StormFire(64, 16, 32,32),
            nn.Upsample(scale_factor=2),
            StormFire(32, 8, 16,16),
            nn.Conv2d(16, 3, kernel_size=3, padding=1),  # Output 3 channels for RGB
            nn.Tanh(),  # Output values between -1 and 1
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.decode(x)


           
        
############ Custom Model ############
class StormFire(nn.Module):
    # This model is based on Fire module from SqueezeNet with the addition of BatchNorm 
    # and the change of ReLU to LeakyReLU

    # autoencoder
    def __init__(
        self,
        inplanes: int,
        squeeze_planes: int,
        expand1x1_planes: int,
        expand3x3_planes: int,
        
    ) -> None:
        super().__init__()
        self.inplanes = inplanes

        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.LeakyReLU(inplace=True)
        self.squeeze_bn = nn.BatchNorm2d(squeeze_planes)
       
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.LeakyReLU(inplace=True)
        self.expand1x1_bn = nn.BatchNorm2d(expand1x1_planes)
        
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.LeakyReLU(inplace=True)
        self.expand3x3_bn = nn.BatchNorm2d(expand3x3_planes)

    def forward(self, x):
        x = self.squeeze_bn(self.squeeze_activation(self.squeeze(x)))
        return torch.cat([
            self.expand1x1_bn(
                self.expand1x1_activation(self.expand1x1(x))
            ),
            self.expand3x3_bn(
                self.expand3x3_activation(self.expand3x3(x)) 
            ),
            ], 1)
            
        


class StormColorModel(nn.Module):
    # This net is a custom version of SqueezeNet: we replaced the Fire module with our custom StormFire module,
    # modified the final convolutional layer into a fully connected layer
    # we used leaky ReLU instead of ReLU
    # we added BatchNorm after each convolutional layer
    # we modified the number of fire modules and the number of filters in each fire module

    def __init__(self, num_classes: int = 251, dropout: float = 0.5) -> None:
        super().__init__()
        self.num_classes = num_classes

        self.features = nn.Sequential(
            
            nn.Conv2d(3, 64, kernel_size=3, stride=2),
            nn.LeakyReLU(inplace=True),

            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            StormFire(64, 16, 64, 64),
            StormFire(128, 16, 64, 64),

            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            StormFire(128, 32, 128, 128),
            StormFire(256, 32, 128, 128),

            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            StormFire(256, 48, 192, 192),
            StormFire(384, 64, 256, 256),
            
        )

     
        self.classifier2 = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 512),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(512, self.num_classes),
        )
       
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        print(x.shape)
        x = self.classifier2(x)
        return torch.flatten(x, 1)


In [8]:
model = StormColorModel()
model.to('cuda')
torchsummary.summary(model, (3, 224, 224))

torch.Size([2, 512, 13, 13])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 111, 111]           1,792
         LeakyReLU-2         [-1, 64, 111, 111]               0
         MaxPool2d-3           [-1, 64, 55, 55]               0
            Conv2d-4           [-1, 16, 55, 55]           1,040
         LeakyReLU-5           [-1, 16, 55, 55]               0
       BatchNorm2d-6           [-1, 16, 55, 55]              32
            Conv2d-7           [-1, 64, 55, 55]           1,088
         LeakyReLU-8           [-1, 64, 55, 55]               0
       BatchNorm2d-9           [-1, 64, 55, 55]             128
           Conv2d-10           [-1, 64, 55, 55]           9,280
        LeakyReLU-11           [-1, 64, 55, 55]               0
      BatchNorm2d-12           [-1, 64, 55, 55]             128
        StormFire-13          [-1, 128, 55, 55]               0
          

In [9]:
x =model.forward(torch.randn(1,3, 224, 224).to('cuda'))
print(x.size())

torch.Size([1, 512, 13, 13])
torch.Size([1, 251])


In [23]:
import torch
import torch.nn as nn

class StormFire(nn.Module):
    def __init__(self, inplanes, squeeze_planes, expand1x1_planes, expand3x3_planes):
        super(StormFire, self).__init__()
        self.inplanes = inplanes

        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.LeakyReLU(inplace=True)
        self.squeeze_bn = nn.BatchNorm2d(squeeze_planes)
       
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.LeakyReLU(inplace=True)
        self.expand1x1_bn = nn.BatchNorm2d(expand1x1_planes)
        
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.LeakyReLU(inplace=True)
        self.expand3x3_bn = nn.BatchNorm2d(expand3x3_planes)

    def forward(self, x):
        x = self.squeeze_bn(self.squeeze_activation(self.squeeze(x)))
        return torch.cat([
            self.expand1x1_bn(
                self.expand1x1_activation(self.expand1x1(x))
            ),
            self.expand3x3_bn(
                self.expand3x3_activation(self.expand3x3(x)) 
            ),
            ], 1)
            
class StormSqueezeNet(nn.Module):
    def __init__(self, num_classes=251, dropout=0.5):
        super(StormSqueezeNet, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2),

        
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            
            StormFire(64, 16, 64, 64),
                       
            StormFire(128, 16, 64, 64),

            nn.MaxPool2d(kernel_size=3, stride=2),

           
            StormFire(128, 32, 128, 128),
            
            StormFire(256, 32, 128, 128),
          
            nn.MaxPool2d(kernel_size=3, stride=2),
            StormFire(256, 48, 192, 192),
            StormFire(384, 64, 256, 256),
        )
    
    def forward(self, x):
        x = self.features(x)
        return x

# Create the model and print the shape of the output
model = StormSqueezeNet()
input_tensor = torch.randn(1, 3, 224, 224)
output = model(input_tensor)
print(output.shape)


torch.Size([1, 512, 13, 13])


In [29]:
import torch
import torch.nn as nn

class StormFire(nn.Module):
    def __init__(self, inplanes, squeeze_planes, expand1x1_planes, expand3x3_planes):
        super(StormFire, self).__init__()
        self.inplanes = inplanes

        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.LeakyReLU(inplace=True)
        self.squeeze_bn = nn.BatchNorm2d(squeeze_planes)
       
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.LeakyReLU(inplace=True)
        self.expand1x1_bn = nn.BatchNorm2d(expand1x1_planes)
        
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.LeakyReLU(inplace=True)
        self.expand3x3_bn = nn.BatchNorm2d(expand3x3_planes)

    def forward(self, x):
        x = self.squeeze_bn(self.squeeze_activation(self.squeeze(x)))
        print(f'After squeeze: {x.shape}')
        x = torch.cat([
            self.expand1x1_bn(
                self.expand1x1_activation(self.expand1x1(x))
            ),
            self.expand3x3_bn(
                self.expand3x3_activation(self.expand3x3(x)) 
            ),
        ], 1)
        print(f'After expand: {x.shape}')
        return x
            
class StormSqueezeNet(nn.Module):
    def __init__(self, num_classes=251, dropout=0.5):
        super(StormSqueezeNet, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1), #added padding
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True), 

            StormFire(64, 16, 64, 64),
            StormFire(128, 16, 64, 64),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True), 

            StormFire(128, 32, 128, 128),
            StormFire(256, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2,ceil_mode=True),

            StormFire(256, 48, 192, 192),
            StormFire(384, 64, 256, 256),
        )
    
    def forward(self, x):
        print(f'Input: {x.shape}')
        for layer in self.features:
            x = layer(x)
            print(f'After {layer.__class__.__name__}: {x.shape}')
        return x

# Create the model and print the shape of the output
model = StormSqueezeNet()
input_tensor = torch.randn(1, 3, 224, 224)
output = model(input_tensor)
print(output.shape)


Input: torch.Size([1, 3, 224, 224])
After Conv2d: torch.Size([1, 64, 112, 112])
After LeakyReLU: torch.Size([1, 64, 112, 112])
After MaxPool2d: torch.Size([1, 64, 56, 56])
After squeeze: torch.Size([1, 16, 56, 56])
After expand: torch.Size([1, 128, 56, 56])
After StormFire: torch.Size([1, 128, 56, 56])
After squeeze: torch.Size([1, 16, 56, 56])
After expand: torch.Size([1, 128, 56, 56])
After StormFire: torch.Size([1, 128, 56, 56])
After MaxPool2d: torch.Size([1, 128, 28, 28])
After squeeze: torch.Size([1, 32, 28, 28])
After expand: torch.Size([1, 256, 28, 28])
After StormFire: torch.Size([1, 256, 28, 28])
After squeeze: torch.Size([1, 32, 28, 28])
After expand: torch.Size([1, 256, 28, 28])
After StormFire: torch.Size([1, 256, 28, 28])
After MaxPool2d: torch.Size([1, 256, 14, 14])
After squeeze: torch.Size([1, 48, 14, 14])
After expand: torch.Size([1, 384, 14, 14])
After StormFire: torch.Size([1, 384, 14, 14])
After squeeze: torch.Size([1, 64, 14, 14])
After expand: torch.Size([1, 512,